#### 【 이미지 기계학습 】

- 이미지 전처리
    * 입력 형태를 맞추는 전처리는 모두 동일 적용
    * 분포를 흔드는 랜덤 변환은 train만 적용
    * train/valid/test 모두 적용
        - 리사이징, 크롭, 배경 제거, 색공간 변경, 픽셀 스케일링 변경
    * train만 진행
        - 데이터 증강용 작업 : Random flip, Random rotation, Color jitter ㅡ Noise 추가  


[1] 모듈 로딩 및 데이터 준비<hr>

In [1]:
## =================================================================
## [1-1] 모듈 로딩
## =================================================================
##- 기본 모듈
import numpy as np
import pandas as pd

##- ML 데이터셋 관련 
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

##- ML 학습 알고리즘 관련
from sklearn.linear_model import LogisticRegression

##- ML 성능지표 관련
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [2]:
## =================================================================
## [1-2] 데이터 준비
## =================================================================
##- 데이터 설정
CSV_PATH = "../Data/csv/mnist_train.csv"

##- 데이터 로딩 
mnistDF  = pd.read_csv(CSV_PATH, header=None)   

[2] 피쳐와 타겟 & 학습용과 테스트용 데이터셋 준비<hr>

In [3]:
## =====================================================
## [2-1] 첫 컬럼: 라벨, 나머지 784개: 픽셀(28x28)
## =====================================================
y = mnistDF.iloc[:, 0].astype(np.int64).values
X = mnistDF.iloc[:, 1:].astype(np.float32).values

print("X shape:", X.shape, "y shape:", y.shape)  # (N, 784), (N,)

X shape: (10001, 784) y shape: (10001,)


In [4]:
## =====================================================
## [2-2] train/test 분리
## =====================================================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


[3] 교차검증 통한 하이퍼파라미터 찾기<hr>

In [5]:
# =========================================================
# [3-1] Pipeline (누수 방지: PCA는 CV fold의 train에서만 fit)
# =========================================================
pipe = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("pca",    PCA(random_state=42)),
    ("clf",    LogisticRegression(max_iter=3000,solver="lbfgs" ))
])

In [6]:
# =========================================================
# [3-2] 교차검증 위한 학습 모델 하이퍼라라미터 
# =========================================================
param_grid = {
    # PCA 차원: 정수(주성분 개수) 또는 누적분산 비율(0~1)
    "pca__n_components": [0.90, 0.95, 0.97, 40, 60, 80, 120],
    # LR 규제 강도(C): 클수록 규제 약함
    "clf__C": [0.1, 1.0, 3.0, 10.0],
}

In [7]:
# =========================================================
# [3-3] 사용자 정의 교차검증 설정
# =========================================================
## 자동 CV는 내부에서 random_state 고정하지 않음 
## 재현성 위해 설정 
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


In [23]:
## ===================================================================================
## [3-4] 교차검증
## -> refit 매개변수 
##      * True  : 전체 train 데이터 + 최적 하이퍼 파라미터로 학습 후 ESM 반환
##      * False : X ===> 개발자가 직접 전체 train 데이터 + 최적 하이퍼 파라미터로 학습 잰행
## ===================================================================================
grid = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    scoring="accuracy",
    cv=cv,
    verbose=3,
    refit=True
)

grid.fit(X_train, y_train)

print("\n[BEST PARAMS]")
print(grid.best_params_)
print("[BEST CV SCORE]", grid.best_score_)

Fitting 5 folds for each of 28 candidates, totalling 140 fits
[CV 1/5] END .clf__C=0.1, pca__n_components=0.9;, score=0.894 total time=   0.7s
[CV 2/5] END .clf__C=0.1, pca__n_components=0.9;, score=0.911 total time=   0.8s
[CV 3/5] END .clf__C=0.1, pca__n_components=0.9;, score=0.918 total time=   0.7s
[CV 4/5] END .clf__C=0.1, pca__n_components=0.9;, score=0.896 total time=   0.7s
[CV 5/5] END .clf__C=0.1, pca__n_components=0.9;, score=0.911 total time=   0.8s
[CV 1/5] END clf__C=0.1, pca__n_components=0.95;, score=0.896 total time=   0.8s
[CV 2/5] END clf__C=0.1, pca__n_components=0.95;, score=0.909 total time=   0.7s
[CV 3/5] END clf__C=0.1, pca__n_components=0.95;, score=0.909 total time=   0.7s
[CV 4/5] END clf__C=0.1, pca__n_components=0.95;, score=0.901 total time=   0.7s
[CV 5/5] END clf__C=0.1, pca__n_components=0.95;, score=0.908 total time=   0.8s
[CV 1/5] END clf__C=0.1, pca__n_components=0.97;, score=0.897 total time=   0.7s
[CV 2/5] END clf__C=0.1, pca__n_components=0.97

[2] PCA만 확인 <hr>

In [14]:
stdScaler = StandardScaler()
scaled_x_trian = stdScaler.fit_transform(X_train)
scaled_x_test = stdScaler.fit_transform(X_test)

print(f'{scaled_x_trian.shape} / {scaled_x_test.shape}')


(8000, 784) / (2001, 784)


In [22]:
pca = PCA(random_state=42, n_components=120)

pca_x_train = pca.fit_transform(scaled_x_trian)
print(f'pca_x_train : {pca_x_train.shape}')

## PCA 모델 파라미터
print(f'n_components                : {pca.n_components}')
print(f'components_                 : {pca.components_}')
print(f'explained_variance_ratio_   : {pca.explained_variance_ratio_.sum()}')




pca_x_train : (8000, 120)
n_components                : 120
components_                 : [[ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [-0. -0. -0. ... -0. -0. -0.]
 ...
 [ 0.  0.  0. ...  0.  0.  0.]
 [-0. -0. -0. ... -0. -0. -0.]
 [ 0.  0.  0. ...  0.  0.  0.]]
explained_variance_ratio_   : 0.8036968111991882
